Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [4]:
import pandas as pd

Задание

1. Загрузите датасет.

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [5]:
data = pd.read_csv("SMSSpamCollection.txt", header = None, sep = "\t", names = ["label", "text"], )

In [6]:
data

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [79]:
labels = list(data.label.values)
texts = list(data.text.values)

In [80]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels)

In [81]:
texts[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [149]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [83]:
v = CountVectorizer()
v.fit(texts)
mx = v.transform(texts)
mx

<5572x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 74169 stored elements in Compressed Sparse Row format>

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [84]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [85]:
result = cross_val_score(LogisticRegression(random_state=2), mx, labels, cv=10, scoring="f1").mean()
result

0.9311542822856882

In [86]:
ans1 ="{:.1f}".format(result)
ans1

'0.9'

In [87]:
f = open("ans1.txt", "w")
f.write(ans1)
f.close
!cat ans1.txt

0.9

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [88]:
sentences = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", "FreeMsg: Txt: claim your reward of 3 hours talk time", "Have you visited the last lecture on physics?", "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", "Only 99$"]

In [89]:
mx1 = v.transform(sentences)
mx1

<5x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 51 stored elements in Compressed Sparse Row format>

In [90]:
regressor = LogisticRegression(random_state=2)
regressor.fit(mx, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [100]:
ans2 = regressor.predict(mx1)
ans2 = str(ans2)[1:-1]
ans2

'1 1 0 0 0'

In [101]:
f1 = open("ans2.txt", "w")
f1.write(str(ans2))
f1.close
!cat ans2.txt

1 1 0 0 0

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [106]:
v2 = CountVectorizer(ngram_range=(2,2))
v2.fit(texts)
mx2 = v2.transform(texts)
mx2

v3 = CountVectorizer(ngram_range=(3,3))
v3.fit(texts)
mx3 = v3.transform(texts)
mx3

v4 = CountVectorizer(ngram_range=(1,3))
v4.fit(texts)
mx4 = v4.transform(texts)
mx4

<5572x104967 sparse matrix of type '<class 'numpy.int64'>'
	with 217549 stored elements in Compressed Sparse Row format>

In [107]:
result2 = cross_val_score(LogisticRegression(random_state=2), mx2, labels, cv=10, scoring="f1").mean()
result2

0.8168951028736983

In [108]:
result3 = cross_val_score(LogisticRegression(random_state=2), mx3, labels, cv=10, scoring="f1").mean()
result3

0.7249691484935827

In [109]:
result4 = cross_val_score(LogisticRegression(random_state=2), mx4, labels, cv=10, scoring="f1").mean()
result4

0.9216545237836128

In [110]:
result2 = "{:.2f}".format(result2)
result2

'0.82'

In [111]:
result3 = "{:.2f}".format(result3)
result3

'0.72'

In [112]:
result4 = "{:.2f}".format(result4)
result4

'0.92'

In [114]:
f2 = open("ans3.txt", "w")
f2.write(str(result2) + " " + str(result3) + " " + str(result4))
f2.close
!cat ans3.txt

0.82 0.72 0.92

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [115]:
from sklearn.naive_bayes import MultinomialNB

In [116]:
bc = MultinomialNB()
bc.fit(mx, labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [144]:
result2b = cross_val_score(MultinomialNB(), mx2, labels, cv=10, scoring="f1").mean()
result2b

0.6457761003991855

In [145]:
result3b = cross_val_score(MultinomialNB(), mx3, labels, cv=10, scoring="f1").mean()
result3b

0.37861105465217015

In [146]:
result4b = cross_val_score(MultinomialNB(), mx4, labels, cv=10, scoring="f1").mean()
result4b

0.888454144419353

In [147]:
type(result2b)

numpy.float64

In [148]:
result2b = "{:.2f}".format(result2b)
result3b = "{:.2f}".format(result3b)
result4b = "{:.2f}".format(result4b)
f3 = open("ans4.txt", "w")
f3.write(str(result2b) + " " + str(result3b) + " " + str(result4b))
f3.close
!cat ans4.txt

0.65 0.38 0.89

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [150]:
tv = TfidfVectorizer()
tv.fit(texts)
tmx = tv.transform(texts)
tmx

<5572x8713 sparse matrix of type '<class 'numpy.float64'>'
	with 74169 stored elements in Compressed Sparse Row format>

In [151]:
result = cross_val_score(LogisticRegression(random_state=2), tmx, labels, cv=10, scoring="f1").mean()
result

0.8520118540708351

In [153]:
f4 = open("ans5.txt", "w")
f4.write(str(-1))
f4.close
!cat ans5.txt

-1